In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

df_original = pd.read_csv('/content/drive/MyDrive/CO544/transformed_rainfall_data.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Assuming your dataframe is named 'df'
df = df_original[~df_original.index.isin([0, 1])]

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming your data is loaded into a DataFrame named 'df'

# Convert the date column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')

# Extracting day of the year as a feature
df['DayOfYear'] = df['Date'].dt.dayofyear

# Select features and target
features = ['DayOfYear']
targets = ['Vavuniya', 'Anuradhapura', 'Maha Illuppallama']

# Feature engineering

# Lag features (previous day's value)
for target in targets:
  df[f'{target}_lag1'] = df[target].shift(1)

# Rolling mean (weekly window)
window_size = 7  # You can experiment with different window sizes
for target in targets:
  df[f'{target}_rolling_mean_{window_size}'] = df[target].rolling(window=window_size).mean()

# Month and Weekday features
# df['Month'] = df['Date'].dt.month
# df['Is_Weekend'] = np.where(df['Date'].dt.weekday_name.isin(['Saturday', 'Sunday']), 1, 0)

# After feature engineering
df.dropna(inplace=True)  # Drop rows with any NaN values

# Splitting the data into training and testing sets
X = df[features + [col for col in df.columns if col.endswith('_lag1') or col.endswith(f'_rolling_mean_{window_size}')]]  # Include engineered features
y_vavuniya = df['Vavuniya']
y_anuradhapura = df['Anuradhapura']
y_maha = df['Maha Illuppallama']



X_train, X_test, y_train_v, y_test_v = train_test_split(X, y_vavuniya, test_size=0.2, random_state=42)
X_train, X_test, y_train_a, y_test_a = train_test_split(X, y_anuradhapura, test_size=0.2, random_state=42)
X_train, X_test, y_train_m, y_test_m = train_test_split(X, y_maha, test_size=0.2, random_state=42)

# ... (rest of your code for training and prediction)


<ipython-input-3-ab9b7845a373>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')
<ipython-input-3-ab9b7845a373>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DayOfYear'] = df['Date'].dt.dayofyear
<ipython-input-3-ab9b7845a373>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [9]:
from sklearn.ensemble import RandomForestRegressor

# Initialize the model
rfr_v = RandomForestRegressor()
rfr_a = RandomForestRegressor()
rfr_m = RandomForestRegressor()

# Fit the model
rfr_v.fit(X_train, y_train_v)
rfr_a.fit(X_train, y_train_a)
rfr_m.fit(X_train, y_train_m)

# Predict
y_pred_v = rfr_v.predict(X_test)
y_pred_a = rfr_a.predict(X_test)
y_pred_m = rfr_m.predict(X_test)

# Evaluate the model
mse_v = mean_squared_error(y_test_v, y_pred_v)
mse_a = mean_squared_error(y_test_a, y_pred_a)
mse_m = mean_squared_error(y_test_m, y_pred_m)

print(f'Random Forest Regressor MSE for Vavuniya: {mse_v}')
print(f'Random Forest Regressor MSE for Anuradhapura: {mse_a}')
print(f'Random Forest Regressor MSE for Maha Illuppallama: {mse_m}')


Random Forest Regressor MSE for Vavuniya: 103.26701261792452
Random Forest Regressor MSE for Anuradhapura: 185.31887045283014
Random Forest Regressor MSE for Maha Illuppallama: 120.6375931698113


##To get the higher efficiency

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
}

# Define RandomForestRegressor instances for each target
rfr_v = RandomForestRegressor(random_state=42)  # For Vavuniya
rfr_a = RandomForestRegressor(random_state=42)  # For Anuradhapura
rfr_m = RandomForestRegressor(random_state=42)  # For Maha Illuppallama

# Perform Grid Search for each target
grid_search_v = GridSearchCV(estimator=rfr_v, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
grid_search_v.fit(X_train, y_train_v)

grid_search_a = GridSearchCV(estimator=rfr_a, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
grid_search_a.fit(X_train, y_train_a)

grid_search_m = GridSearchCV(estimator=rfr_m, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
grid_search_m.fit(X_train, y_train_m)

# Get the best parameters
best_params_v = grid_search_v.best_params_
best_params_a = grid_search_a.best_params_
best_params_m = grid_search_m.best_params_

# Use the best parameters to initialize the final model
rfr_v = RandomForestRegressor(**best_params_v)
rfr_a = RandomForestRegressor(**best_params_a)
rfr_m = RandomForestRegressor(**best_params_m)

# Fit the models with the best parameters
rfr_v.fit(X_train, y_train_v)
rfr_a.fit(X_train, y_train_a)
rfr_m.fit(X_train, y_train_m)

# Predict with the tuned models
y_pred_v = rfr_v.predict(X_test)
y_pred_a = rfr_a.predict(X_test)
y_pred_m = rfr_m.predict(X_test)

# Evaluate the tuned models
mse_v = mean_squared_error(y_test_v, y_pred_v)
mse_a = mean_squared_error(y_test_a, y_pred_a)
mse_m = mean_squared_error(y_test_m, y_pred_m)

print(f'Tuned Random Forest Regressor MSE for Vavuniya: {mse_v}')
print(f'Tuned Random Forest Regressor MSE for Anuradhapura: {mse_a}')
print(f'Tuned Random Forest Regressor MSE for Maha Illuppallama: {mse_m}')


Fitting 3 folds for each of 648 candidates, totalling 1944 fits
Fitting 3 folds for each of 648 candidates, totalling 1944 fits
Fitting 3 folds for each of 648 candidates, totalling 1944 fits
Tuned Random Forest Regressor MSE for Vavuniya: 95.6342689670009
Tuned Random Forest Regressor MSE for Anuradhapura: 121.12621125101145
Tuned Random Forest Regressor MSE for Maha Illuppallama: 112.71575172264795
